Dans ce TD, nous allons implémenter une collecte simple de données via une API : https://rapidapi.com/edamam/api/edamam-food-and-grocery-database. Cela permet d'enrichir des jeux de données d'images lorsque notre problématique requiert par exemple des images spécifiques.

Le but est d'obtenir une extraction des produits à base de champagne dans un fichier “.csv”, contenant pour chaque produit les données suivantes : foodId, label, category, foodContentsLabel, image, ainsi que les images correspondantes.

Rassembler les imports dans la cellule suivante.

In [2]:
import os

from dotenv import load_dotenv
import pandas as pd
import requests

La communication avec une API implique de s'identifier. Cela se fait par l'intermédiaire d'une clef. C'est une bonne pratique de ne pas laisser tout ce qui touche à l'identification en dur dans le code si celui-ci est amené à être partagé. Il existe plusieurs manières d'éviter cet écueil, en voici un bon résumé : https://blog.gitguardian.com/how-to-handle-secrets-in-python/.

Ici, pour accélérer la mise en place et ne pas vous faire créer un compte, je vous partage une clef API d'un compte test. Pour autant, nous suivrons le principe de séparer la clef du code en utilisant la méthode 1 du résumé, à savoir par l'intermédiaire d'un fichier .env. ainsi que la librairie dotenv (nom du module), qui nécessitera une installation (nom du package : python-dotenv).



Récupérer les variables d'environnement du fichier .env en appelant load_dotenv(). Utiliser os.getenv pour stocker la valeur de la clef API dans une variable. Vérifier TEMPORAIREMENT que la clef a bien été chargée en l'affichant.

In [37]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

En observant la requête exemple dans "Code Snippets", préparer les champs d'une requête permettant de récolter les produits appartenant au type de nutrition "cooking" et contenant au moins l'ingrédient "champagne". Trouver le nommage exact des paramètres à renseigner dans la requête grâce à la documentation des paramètres optionnels de l'API.

In [6]:
url = "https://edamam-food-and-grocery-database.p.rapidapi.com/api/food-database/v2/parser"

params = {
    "nutrition-type": 'cooking',
    "ingr": "champagne",
}

headers = {
    "x-rapidapi-key": API_KEY,
    "x-rapidapi-host": "edamam-food-and-grocery-database.p.rapidapi.com",
}

A l'aide de la bibliothèque requests, envoyer la requête GET correspondante à l'API. Stocker l'output dans une variable et en faire un affichage.

In [7]:
response = requests.get(
    url=url,
    headers=headers,
    params=params,
)

In [13]:
response.json()["hints"]

[{'food': {'foodId': 'food_a656mk2a5dmqb2adiamu6beihduu',
   'uri': 'http://www.edamam.com/ontologies/edamam.owl#Food_table_white_wine',
   'label': 'Champagne',
   'knownAs': 'dry white wine',
   'nutrients': {'ENERC_KCAL': 82.0,
    'PROCNT': 0.07,
    'FAT': 0.0,
    'CHOCDF': 2.6,
    'FIBTG': 0.0},
   'category': 'Generic foods',
   'categoryLabel': 'food',
   'image': 'https://www.edamam.com/food-img/a71/a718cf3c52add522128929f1f324d2ab.jpg'},
  'measures': [{'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_serving',
    'label': 'Serving',
    'weight': 147.0},
   {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_unit',
    'label': 'Whole',
    'weight': 750.0},
   {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_splash',
    'label': 'Splash',
    'weight': 84.0},
   {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_gram',
    'label': 'Gram',
    'weight': 1.0},
   {'uri': 'http://www.edamam.com/ontologies/edamam.owl#Measure_ounce'

La fonction print renvoie le code de la réponse, permettant de confirmer qu'elle fut un succès ou qu'elle a rencontré une erreur. Accéder désormais au contenu de la réponse en utilisant la méthode .json().

In [ ]:
# Votre code ici

Reconstruire le dataframe correspondant aux données qui nous intéressent (pour rappel, seuls les champs suivants nous intéressent : foodId, label, category, foodContentsLabel, image).

In [27]:
%%time
selected_cols = ["foodId", "label", "category", "foodContentsLabel", "image"]
filtered_response = [
    {
        col: elt["food"].get(col)
        for col in selected_cols
    }
    for elt in response.json()["hints"]
]
df_extract = pd.DataFrame(data=filtered_response)

CPU times: total: 0 ns
Wall time: 488 µs


In [28]:
%%time
selected_cols = ["foodId", "label", "category", "foodContentsLabel", "image"]
df_extract = pd.DataFrame(
    data=[x["food"] for x in response.json()["hints"]],
    columns=selected_cols,
)

CPU times: total: 0 ns
Wall time: 1.24 ms


Afficher les 10 premiers éléments du dataframe ainsi que sa taille totale.

In [36]:
df_extract.head(10)

,foodId,label,category,foodContentsLabel,image
0,food_a656mk2a5dmqb2adiamu6beihduu,Champagne,Generic foods,NaN,https://www.edamam.com/food-img/a71/a718cf3c52...
1,food_b753ithamdb8psbt0w2k9aquo06c,"Champagne Vinaigrette, Champagne",Packaged foods,OLIVE OIL; BALSAMIC VINEGAR; CHAMPAGNE VINEGAR...,NaN
2,food_b3dyababjo54xobm6r8jzbghjgqe,"Champagne Vinaigrette, Champagne",Packaged foods,INGREDIENTS: WATER; CANOLA OIL; CHAMPAGNE VINE...,https://www.edamam.com/food-img/d88/d88b64d973...
3,food_a9e0ghsamvoc45bwa2ybsa3gken9,"Champagne Vinaigrette, Champagne",Packaged foods,CANOLA AND SOYBEAN OIL; WHITE WINE (CONTAINS S...,NaN
4,food_an4jjueaucpus2a3u1ni8auhe7q9,"Champagne Vinaigrette, Champagne",Packaged foods,WATER; CANOLA AND SOYBEAN OIL; WHITE WINE (CON...,NaN
5,food_bmu5dmkazwuvpaa5prh1daa8jxs0,"Champagne Dressing, Champagne",Packaged foods,SOYBEAN OIL; WHITE WINE (PRESERVED WITH SULFIT...,https://www.edamam.com/food-img/ab2/ab2459fc2a...
6,food_alpl44taoyv11ra0lic1qa8xculi,Champagne Buttercream,Generic meals,sugar; butter; shortening; vanilla; champagne;...,NaN
7,food_am5egz6aq3fpjlaf8xpkdbc2asis,Champagne Truffles,Generic meals,butter; cocoa; sweetened condensed milk; vanil...,NaN
8,food_bcz8rhiajk1fuva0vkfmeakbouc0,Champagne Vinaigrette,Generic meals,champagne vinegar; olive oil; Dijon mustard; s...,NaN
9,food_a79xmnya6togreaeukbroa0thhh0,Champagne Chicken,Generic meals,"Flour; Salt; Pepper; Boneless, Skinless Chicke...",NaN


Pour certaines lignes, l'image n'est pas disponible et dans ce cas l'URL est à None. Afficher le nombre de lignes dont l'image dispose d'une URL.

In [ ]:
# Votre code ici

In [ ]:
# Votre code ici

Enregistrer cette extraction au format csv dans le dossier courant.

In [35]:
df_extract.to_csv("extract.csv")

Finalement, récupérer les images dont l'URL est renseignée, à l'aide de .content. Les sauvegarder dans le dossier "images". Vérifier que le nombre de fichiers téléchargés correspond bien au nombre d'images attendu.

In [62]:
num_img = 0
for url in df_extract["image"]:
   if not pd.isnull(url):
        img_data = requests.get(url=url).content
        with open(
            os.path.join("images", f"champagne_{num_img}.jpg"),
            "wb",
        ) as f:
            f.write(img_data)
        num_img += 1